In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import necessary package
import tensorflow as tf
from tensorflow import keras

import sklearn
import sklearn.model_selection
from sklearn import preprocessing

# 畫圖表用
import matplotlib.pyplot as plt

In [ ]:
# read the data
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
# 由此看出資料為灰階，最大值應為255，需做標準化(normalization)
train.describe()

In [ ]:
test.head()

In [ ]:
# data preprocessing
# split the data and label
train_y = train["label"]
train_x = train.drop(labels = ["label"], axis = 1)

# normalization
train_x = train_x / 255.0
test = test / 255.0

# reshape
train_x = train_x.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

# label(one hot encoding)
train_y = pd.get_dummies(train_y)

In [ ]:
# split the data to prevent ImageDataGenerator modify validation data
train_x, val_x, train_y, val_y = sklearn.model_selection.train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [ ]:
# Adopt other model
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 5, activation="relu", padding="same", input_shape=[28, 28, 1]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, 5, activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2),
    keras.layers.Dropout(0.25),
    # 兩個3x3
    keras.layers.Conv2D(64, 3, activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, 3, activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2),
    keras.layers.Dropout(0.25),
    keras.layers.Conv2D(64, 3, activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2),
    keras.layers.Dropout(0.25),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(10, activation="softmax")
])

optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
# without data augmentation
# early_stopping =  keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# model_result = model.fit(train_x, train_y, batch_size=32, epochs=100, validation_data=(val_x, val_y), shuffle=True, callbacks=[early_stopping])

In [ ]:
# with data augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# prevent ImageDataGenerator modify validation data
val_datagen = keras.preprocessing.image.ImageDataGenerator()

train_datagen.fit(train_x)
val_datagen.fit(val_x)

# Early Stopping
early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# train the model
model_result = model.fit(
    train_datagen.flow(train_x, train_y, batch_size=128),
    validation_data=val_datagen.flow(val_x, val_y, batch_size=128),
    epochs=100,
    callbacks=[early_stopping]
)

In [ ]:
plt.figure(figsize=(30, 10))

plt.subplot(1, 2, 1)
plt.plot(model_result.history["loss"], label="training")
plt.plot(model_result.history["val_loss"], label="validation")
#plt.axhline(0.55, c="red", linestyle="--")
#plt.axhline(0.35, c="yellow", linestyle="--")
#plt.axhline(0.15, c="green", linestyle="--")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(model_result.history["accuracy"], label="training")
plt.plot(model_result.history["val_accuracy"], label="validation")
#plt.axhline(0.75, c="red", linestyle="--")
#plt.axhline(0.80, c="green", linestyle="--")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.show()

In [ ]:
# Inference
results = model.predict(test)

In [ ]:
# 選最大可能性的值做為label
results = np.argmax(results, axis = 1)

results = pd.Series(results, name="label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"), results], axis = 1)

submission.to_csv("submission.csv", index=False)